##  準備

In [2]:
from pathlib import Path
from bs4 import BeautifulSoup
import json
import os
import re
import time
import logging
import pandas as pd
!pip install selenium
!pip install beautiful requests
!pip install BeautifulSoup4
from selenium import webdriver
from selenium.webdriver.support.select import Select
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert

pd.set_option('display.max_rows', 100)

  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
requests 2.22.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.16 which is incompatible.
cookiecutter 1.7.2 requires requests>=2.23.0, but you have requests 2.22.0 which is incompatible.
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
ERROR: pip's dependency resolv

In [57]:
search_url = "https://suzukadaigaku-web.campusplan.jp/public/web/Syllabus/WebSyllabusKensaku/UI/WSL_SyllabusKensaku.aspx"
base_url = "https://suzukadaigaku-web.campusplan.jp"
browser1 = webdriver.Chrome()
browser1.get(search_url)
sleep(5)
search_button = browser1.find_element(By.NAME,'btnSearch')
search_button.click()

In [55]:
#検索結果の表示ページを切り替える
def nexe_page(browser):
    next_button = browser1.find_element(By.LINK_TEXT, '次ページ＞')   
    next_button.click()


def is_next_page_exist(browser):
    next_button = browser1.find_element(By.LINK_TEXT, '次ページ＞')   
    if(next_button.get_attribute("disabled") == "true"):
        print("this is the last page.")
        return False
    else:
        return True

def record_links(browser):
    main_table = browser1.find_element(By.CLASS_NAME, 'last-table')
    tr_elements = main_table.find_elements(By.TAG_NAME, 'tr')
    for tr_element in tr_elements:
    # <input>要素を取得します
        try:
            input_element = tr_element.find_element(By.TAG_NAME, 'input')
        except:
            #ヘッダー行の回避
            continue
        raw_code = input_element.get_attribute("onClick")
        start_index = raw_code.find("win = window.open('") + 1
        end_index = raw_code.find("','WSL_SyllabusSanshoaspx")
        raw_code = input_element.get_attribute("onClick")
        start_index = raw_code.find("win = window.open('") + 1
        end_index = raw_code.find("','WSL_SyllabusSanshoaspx")
        substring = raw_code[start_index:end_index]
        key = substring.replace("in = window.open('", "")
        access_url = base_url + key
        with open('links.txt', 'a') as f:
            print(access_url, file=f)


In [56]:
record_links(browser1)
while(is_next_page_exist(browser1)):
    nexe_page(browser1)
    record_links(browser1)

this is the last page.


In [7]:
#表示されているページにある<a>タグを全て取得しそのリンクにアクセスしhtmlをダウンロードする関数
def get_html(save_path):

    

    # 保存先の作成
    def create_directory_if_not_exists(directory_path):
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
            print(f"ディレクトリ '{directory_path}' を作成しました。")
            logging.info(f"ディレクトリ '{directory_path}' を作成しました。")   
        else:
            print(f"ディレクトリ '{directory_path}' は既に存在します。")



    

    main_table = browser1.find_element(By.CLASS_NAME, 'last-table')
    tr_elements = main_table.find_elements(By.TAG_NAME, 'tr')
    count = 0
    for tr_element in tr_elements:
        # <input>要素を取得します
        count = count+1
        try:
            input_element = tr_element.find_element(By.TAG_NAME, 'input')
        except:
            continue
        td_elements = tr_element.find_elements(By.TAG_NAME, 'td')
        td_elements_text = []
        for td_element in td_elements:
            td_elements_text.append(td_element.text)

        try:
            fixed_kubun = td_elements_text[9]
            directory_path = "./html/" + fixed_kubun
            create_directory_if_not_exists(directory_path)

            raw_code = input_element.get_attribute("onClick")
            start_index = raw_code.find("win = window.open('") + 1
            end_index = raw_code.find("','WSL_SyllabusSanshoaspx")
            substring = raw_code[start_index:end_index]
            key = substring.replace("in = window.open('", "")
            url = "https://narapu-web.campusplan.jp/public/web/Syllabus/WebSyllabusSansho/UI/WSL_SyllabusSansho.aspx" + key
            response = requests.get(url)
            html = response.text
            time.sleep(3)
            # htmlをファイルに保存
            file_name = td_elements_text[2]
            with open(r""+directory_path + "/" + file_name + ".html", 'w', encoding='utf-8') as f:
                f.write(html)
                print(file_name)
                logging.info("complete downloading" +file_name)
        except AttributeError as e:

            print("second_exception")
            print(e)
            pass


In [8]:
get_html("/")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".last-table"}
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x000000010f277598 chromedriver + 4973976
1   chromedriver                        0x000000010f26e913 chromedriver + 4938003
2   chromedriver                        0x000000010ee2bcd6 chromedriver + 470230
3   chromedriver                        0x000000010ee6f2ff chromedriver + 746239
4   chromedriver                        0x000000010ee6f591 chromedriver + 746897
5   chromedriver                        0x000000010eeb1384 chromedriver + 1016708
6   chromedriver                        0x000000010ee939ed chromedriver + 895469
7   chromedriver                        0x000000010eeae856 chromedriver + 1005654
8   chromedriver                        0x000000010ee93793 chromedriver + 894867
9   chromedriver                        0x000000010ee6121f chromedriver + 688671
10  chr

In [102]:
browser1.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="dd0ffd2aae840de37fec0845f61a1296")>>